In [ ]:
# default_exp row_model

In [ ]:
#exporti 

from pydantic_pandas.default_standard_lib import *
from pydantic_pandas.core import BaseModel,DataFrame
from pandas import notnull
from pydantic import ValidationError
from pydantic.utils import update_not_none

# Row Model

In [ ]:
#export

class RowModel(BaseModel):
    class Config:
        on_errors: Literal[
            'skip',
            'raise',
            'coerce',
        ] = 'raise'
        use_alias_for_columns:bool=False

In [ ]:
#exporti

def parse_dataframe_records(
    row_model:Type[RowModel],
    df:DataFrame
)->DataFrame:
    # replace NaN with None
    df = df.where(notnull(df), None)
    records = df.to_dict('records')
    parsed = []
    for record in records:
        try:
            parsed_record = row_model.parse_obj(record)
        except ValidationError as e:
            on_errors = getattr(
                row_model.Config,
                'on_errors',
                'raise'
            )
            if on_errors=='skip':
                continue
            if on_errors=="coerce":
                parsed_record = row_model.construct(record)
            if on_errors=='raise':
                raise
        parsed.append(parsed_record)
    by_alias = getattr(
        row_model.Config,
        'use_alias_for_columns',
        False
    )
    parsed_records = [
        row.dict(by_alias=by_alias) for row in parsed
    ]
    df = pd.DataFrame.from_records(parsed_records)
    return DataFrame.validate(df)
                
            
        

In [ ]:
from pydantic_pandas.utils import dummy_df

class Record(RowModel):
    string:str
    number: int

df = dummy_df(Record)

assert not parse_dataframe_records(
    Record,
    df
).empty

In [ ]:
#export 

class TypedRecordFrame(DataFrame):
    row_model: Optional[Type[BaseModel]] = None
        
    @classmethod
    def __get_validators__(cls):

        yield cls.validate_rows
    
    
    @classmethod
    def __modify_schema__(cls, field_schema: Dict[str, Any]) -> None:
        row_model=None
        if cls.row_model:
            row_model = cls.row_model.schema()
        update_not_none(
            field_schema,
            row_model=row_model
        )
    
    @classmethod
    def validate_rows(cls, df):
        
        return parse_dataframe_records(
            cls.row_model,
            df
        )


In [ ]:
#exporti

class RecordFrameMeta(type):
    def __getitem__(self, constraint):

        return type('RecordFrame', (TypedRecordFrame,), {'row_model': constraint})
        

In [ ]:
#export

class RecordFrame(DataFrame, metaclass=RecordFrameMeta):
    pass

In [ ]:
#export

def recordframe(
    *,
    row_model: Type[BaseModel] = None
) -> Type[RecordFrame]:
    # use kwargs then define conf in a dict to aid with IDE type hinting
    namespace = dict(row_model=row_model)
    return type('RecordFrame', (TypedRecordFrame,), namespace)

In [ ]:
#export 

def record_model(model:Type[RowModel]):
    return RecordFrame[model]

In [ ]:
class Record(RowModel):
    string:str
    number:int

In [ ]:
df = dummy_df(Record)

In [ ]:
RecordFrame[Record].validate_rows(df).head()

,string,number
0,,0
1,0,0
2,,0
3,0,0
4,,0


In [ ]:
recordframe(
    row_model=Record
).validate_rows(df).head()

,string,number
0,,0
1,0,0
2,,0
3,0,0
4,,0


In [ ]:
@record_model
class Record(BaseModel):
    string: str
    number: int

Record.validate_rows(df).head()

,string,number
0,,0
1,0,0
2,,0
3,0,0
4,,0


In [ ]:
#hide
!nbdev_build_lib

Converted 00_core.ipynb.
Converted 01_row_model.ipynb.
Converted 98_utils.ipynb.
Converted 99_default_standard_lib.ipynb.
Converted index.ipynb.
